# Installing Whisper

The commands below will install the Python packages needed to use Whisper models and evaluate the transcription results.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import zipfile
zip_file = "/content/drive/My Drive/kathbath.zip"
extract_dir = 'folder'
with zipfile.ZipFile(zip_file,'r') as zip_ref:
  zip_ref.extractall(extract_dir)

In [ ]:
# import urllib.request

# url = "https://objectstore.e2enetworks.net/indic-asr-public/indicwhisper/vistaar/kathbath.zip"
# file_name = "kathbath.zip"

# urllib.request.urlretrieve(url, file_name)

('kathbath.zip', <http.client.HTTPMessage at 0x7856403bdcf0>)

In [ ]:
# import zipfile
# zip_file = "kathbath.zip"

# with zipfile.ZipFile(zip_file, 'r') as zip_ref:
#     for file in zip_ref.namelist():
#         if 'hindi/' in file:
#             zip_ref.extract(file)

In [ ]:
# folder_to_zip = 'kathbath'
# zip_file_path = 'folder/kathbath.zip'

# shutil.make_archive(zip_file_path, 'zip', folder_to_zip)

In [ ]:
# source_folder_path = 'folder/kathbath.zip.zip'
# destination = '/content/drive/My Drive/folder'

# shutil.copytree(source_folder_path,destination)

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install setuptools-rust
!apt-get -qq install -y ffmpeg

In [36]:
import os
import numpy as np
import jiwer
try:
    import tensorflow
except ImportError:
    pass
import torch
import pandas as pd
import whisper
import torchaudio
from tqdm.notebook import tqdm
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [23]:
os.listdir()

['.config',
 'folder',
 '.ipynb_checkpoints',
 'drive',
 'Audio Files',
 'sample_data']

In [18]:
folder_path = 'Audio Files'
audio_files_count = 0

for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.wav') or file.endswith('.mp3') or file.endswith('.flac'):
            audio_files_count += 1

In [24]:
df = pd.read_csv('folder/hindi/test/bucket.csv')
df

,file_path,transcript,length
0,/nlsasfs/home/ai4bharat/gramesh/speechteam/e2e...,सुधीर मिश्रा की ये फिल्म लेखक शरत चंद्र चट्टोप...,92137
1,/nlsasfs/home/ai4bharat/gramesh/speechteam/e2e...,खोज कम मांग के लिए इंजन में वेबसाइट पदोन्नति,74304
2,/nlsasfs/home/ai4bharat/gramesh/speechteam/e2e...,ट्रेन लक्ष्मीबाई नगर भी नहीं पहुंचती कि लोगों ...,92137
3,/nlsasfs/home/ai4bharat/gramesh/speechteam/e2e...,नई पॉलिसी के तहत एक्सट्रा चार्ज देकर ज्यादा एफ...,138577
4,/nlsasfs/home/ai4bharat/gramesh/speechteam/e2e...,उन्होंने वन मुख्यालय से प्रस्ताव पर अनुमति प्र...,79877
...,...,...,...
1924,/nlsasfs/home/ai4bharat/gramesh/speechteam/e2e...,वे इसके लिए कार्यशालाएं भी आयोजित करते हैं,70589
1925,/nlsasfs/home/ai4bharat/gramesh/speechteam/e2e...,नर की बिना बालों वाली पूंछ होती है,57957
1926,/nlsasfs/home/ai4bharat/gramesh/speechteam/e2e...,रहुघाट पर पन बिजली परियोजना प्रस्तावित की गई है,85450
1927,/nlsasfs/home/ai4bharat/gramesh/speechteam/e2e...,विधिक कहावत यह है कि कानून के प्रति अनभिज्ञता ...,95852


In [25]:
df['file_path'] = df['file_path'].astype(str)
df['file_path'] = df['file_path'].apply(lambda x: x.split('/')[-1])
df

,file_path,transcript,length
0,844424932919645-1202-f.wav,सुधीर मिश्रा की ये फिल्म लेखक शरत चंद्र चट्टोप...,92137
1,844424932410060-267-f.wav,खोज कम मांग के लिए इंजन में वेबसाइट पदोन्नति,74304
2,844424930954560-564-f.wav,ट्रेन लक्ष्मीबाई नगर भी नहीं पहुंचती कि लोगों ...,92137
3,844424930627604-153-f.wav,नई पॉलिसी के तहत एक्सट्रा चार्ज देकर ज्यादा एफ...,138577
4,844424930966728-564-f.wav,उन्होंने वन मुख्यालय से प्रस्ताव पर अनुमति प्र...,79877
...,...,...,...
1924,844424933507454-540-m.wav,वे इसके लिए कार्यशालाएं भी आयोजित करते हैं,70589
1925,844424933474107-540-m.wav,नर की बिना बालों वाली पूंछ होती है,57957
1926,844424933481727-288-m.wav,रहुघाट पर पन बिजली परियोजना प्रस्तावित की गई है,85450
1927,844424933454608-288-m.wav,विधिक कहावत यह है कि कानून के प्रति अनभिज्ञता ...,95852


In [26]:
df = df.rename(columns={'transcript':'original_transcript'})
df

,file_path,original_transcript,length
0,844424932919645-1202-f.wav,सुधीर मिश्रा की ये फिल्म लेखक शरत चंद्र चट्टोप...,92137
1,844424932410060-267-f.wav,खोज कम मांग के लिए इंजन में वेबसाइट पदोन्नति,74304
2,844424930954560-564-f.wav,ट्रेन लक्ष्मीबाई नगर भी नहीं पहुंचती कि लोगों ...,92137
3,844424930627604-153-f.wav,नई पॉलिसी के तहत एक्सट्रा चार्ज देकर ज्यादा एफ...,138577
4,844424930966728-564-f.wav,उन्होंने वन मुख्यालय से प्रस्ताव पर अनुमति प्र...,79877
...,...,...,...
1924,844424933507454-540-m.wav,वे इसके लिए कार्यशालाएं भी आयोजित करते हैं,70589
1925,844424933474107-540-m.wav,नर की बिना बालों वाली पूंछ होती है,57957
1926,844424933481727-288-m.wav,रहुघाट पर पन बिजली परियोजना प्रस्तावित की गई है,85450
1927,844424933454608-288-m.wav,विधिक कहावत यह है कि कानून के प्रति अनभिज्ञता ...,95852


In [27]:
df.rename(columns={'file_path':'file_id'},inplace=True)
df

,file_id,original_transcript,length
0,844424932919645-1202-f.wav,सुधीर मिश्रा की ये फिल्म लेखक शरत चंद्र चट्टोप...,92137
1,844424932410060-267-f.wav,खोज कम मांग के लिए इंजन में वेबसाइट पदोन्नति,74304
2,844424930954560-564-f.wav,ट्रेन लक्ष्मीबाई नगर भी नहीं पहुंचती कि लोगों ...,92137
3,844424930627604-153-f.wav,नई पॉलिसी के तहत एक्सट्रा चार्ज देकर ज्यादा एफ...,138577
4,844424930966728-564-f.wav,उन्होंने वन मुख्यालय से प्रस्ताव पर अनुमति प्र...,79877
...,...,...,...
1924,844424933507454-540-m.wav,वे इसके लिए कार्यशालाएं भी आयोजित करते हैं,70589
1925,844424933474107-540-m.wav,नर की बिना बालों वाली पूंछ होती है,57957
1926,844424933481727-288-m.wav,रहुघाट पर पन बिजली परियोजना प्रस्तावित की गई है,85450
1927,844424933454608-288-m.wav,विधिक कहावत यह है कि कानून के प्रति अनभिज्ञता ...,95852


In [28]:
df['whisper_transcript'] = ''
df['WER'] = 0.0

In [29]:
df

,file_id,original_transcript,length,whisper_transcript,WER
0,844424932919645-1202-f.wav,सुधीर मिश्रा की ये फिल्म लेखक शरत चंद्र चट्टोप...,92137,,0.0
1,844424932410060-267-f.wav,खोज कम मांग के लिए इंजन में वेबसाइट पदोन्नति,74304,,0.0
2,844424930954560-564-f.wav,ट्रेन लक्ष्मीबाई नगर भी नहीं पहुंचती कि लोगों ...,92137,,0.0
3,844424930627604-153-f.wav,नई पॉलिसी के तहत एक्सट्रा चार्ज देकर ज्यादा एफ...,138577,,0.0
4,844424930966728-564-f.wav,उन्होंने वन मुख्यालय से प्रस्ताव पर अनुमति प्र...,79877,,0.0
...,...,...,...,...,...
1924,844424933507454-540-m.wav,वे इसके लिए कार्यशालाएं भी आयोजित करते हैं,70589,,0.0
1925,844424933474107-540-m.wav,नर की बिना बालों वाली पूंछ होती है,57957,,0.0
1926,844424933481727-288-m.wav,रहुघाट पर पन बिजली परियोजना प्रस्तावित की गई है,85450,,0.0
1927,844424933454608-288-m.wav,विधिक कहावत यह है कि कानून के प्रति अनभिज्ञता ...,95852,,0.0


In [17]:
# import os
# import shutil

# main_folder = 'folder/hindi/test/audio/'

# new_folder = 'Audio Files'

# for root, dirs, files in os.walk(main_folder):
#     for file in files:
#         if file.endswith('.wav') or file.endswith('.mp3') or file.endswith('.flac'):
#             src_path = os.path.join(root, file)
#             dst_path = os.path.join(new_folder, file)
#             shutil.move(src_path, dst_path)

In [ ]:
a = df.iloc[0,0]

In [30]:
model = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [00:27<00:00, 55.2MiB/s]


In [31]:
demo = df.copy()

In [32]:
file_ids = list(demo.loc[:,'file_id'])


In [33]:
result = model.transcribe(os.path.join("Audio Files","844424932919645-1202-f.wav"),language='hi')
print(result['text'])

 सुदिर मिश्रा की ये फिर्म लेकिक शर्च चंटर चटोपाध्या के पन्यास देविदास पर आधारित होगी।


In [49]:
directory = 'Audio Files'
files = os.listdir(directory)

for file in tqdm(files, desc="Processing", unit="file"):
    result = model.transcribe(os.path.join(directory, file), language='hi')
    demo.loc[demo['file_id'] == file, 'whisper_transcript'] = result['text']
    demo.loc[demo['file_id'] == file, 'WER'] = jiwer.wer(demo.loc[demo['file_id'] == file, 'original_transcript'].values[0], result['text'])

Processing:   0%|          | 0/1929 [00:00<?, ?file/s]

In [50]:
demo.loc[demo['file_id']=='844424932563153-769-m.wav']


,file_id,original_transcript,length,whisper_transcript,WER
1259,844424932563153-769-m.wav,इससे पहले गृहमंत्री सिंह के आवास पर मीटिंग हुई,47926,इसे पहल गरी मंत्री इसे हैं के अवास पर मीटिंग ...,0.888889


In [104]:
demo

,file_id,original_transcript,length,whisper_transcript,WER
0,844424932919645-1202-f.wav,सुदिर मिश्रा की ये फिर्म लेकिक शर्च चंटर चटोप...,92137,सुदिर मिश्रा की ये फिर्म लेकिक शर्च चंटर चटोप...,0.0
1,844424932410060-267-f.wav,खोज कम मांग के लिए इंजन में वेबसाइट पदोन्नति,74304,खोज कम मांग के लिए इंज़ण में वेबसाइट पदोनती,0.222222
2,844424930954560-564-f.wav,ट्रेन लक्ष्मीबाई नगर भी नहीं पहुंचती कि लोगों ...,92137,ट्रेन लक्षमीबाई नगर भी नहीं पोंषती कि लोगो का...,0.428571
3,844424930627604-153-f.wav,नई पॉलिसी के तहत एक्सट्रा चार्ज देकर ज्यादा एफ...,138577,नहीं पालिसी के तहत एक्स्टर चार्ज देकर जादा एफ...,0.428571
4,844424930966728-564-f.wav,उन्होंने वन मुख्यालय से प्रस्ताव पर अनुमति प्र...,79877,वोनोंने वन मुख्याले से प्रस्ताव पर अनुमती प्र...,0.384615
...,...,...,...,...,...
1924,844424933507454-540-m.wav,वे इसके लिए कार्यशालाएं भी आयोजित करते हैं,70589,वे इसके लिए कारे साले भी आयोजित करते हैं।,0.375
1925,844424933474107-540-m.wav,नर की बिना बालों वाली पूंछ होती है,57957,नार के बिना बालो वाली पूछ होती हैं।,0.625
1926,844424933481727-288-m.wav,रहुघाट पर पन बिजली परियोजना प्रस्तावित की गई है,85450,रागु घाट पर पन भीजली पर्योजना परस्तावित की गई...,0.666667
1927,844424933454608-288-m.wav,विधिक कहावत यह है कि कानून के प्रति अनभिज्ञता ...,95852,वेधिक कहावत यह है कि कानून के प्रति अनवेदगिरत...,0.230769
